### Training

In [1]:
# loading libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import math

# load the dataset
file_path = '/kaggle/input/spotify-million-song-dataset/spotify_millsongdata.csv'
# call the data frame 'df'
df = pd.read_csv(file_path)

# hyperparameters
batch_size = 64
block_size = 128
max_iters = 15000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 128
n_head = 4
n_layer = 4
dropout = 0.0

# concatenates all the strings into a single string, with each original string separated by a newline character
text = df['text'].str.cat(sep='\n')

# text tokenize function
def text_tokenize(text):
    # identifying unique characters and creating mappings
    chars = sorted(list(set(text)))
    vocab_size = len(chars)
    stoi = {ch: i for i, ch in enumerate(chars)}
    itos = {i: ch for i, ch in enumerate(chars)}

    # encoding function
    encode = lambda s: [stoi[c] for c in s]

    # decoding function
    decode = lambda l: ''.join([itos[i] for i in l])

    # apply encoding to the text
    data = torch.tensor(encode(text), dtype=torch.long)
    
    return data, stoi, itos, vocab_size, encode, decode

# apply the tokenize function
data, stoi, itos, vocab_size, encode, decode = text_tokenize(text)

# data split
# splits the data into train and validation sets.
def train_test_split(data, train_ratio):
    n = int(train_ratio * len(data))
    train_data = data[:n]
    val_data = data[n:]
    return train_data, val_data
# split the dataset into 80% as training and 20% as validation
train_data, val_data = train_test_split(data, 0.8)

# batch function
def get_batch(split):
    # choose the data source
    data = train_data if split == 'train' else val_data
    # randomly select batch_size starting indices for the batch
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # generate input sequences (x) by selecting blocks of block_size length starting from the randomly selected indices
    x = torch.stack([data[i:i+block_size] for i in ix])
    # generate target sequences (y) by selecting the next block_size characters for each input sequence
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # move the input and target tensors to the specified device
    x, y = x.to(device), y.to(device)

    return x, y

# loss estimation
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# model component
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

# training loop
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
# save the model
torch.save(model.state_dict(), 'lyric_generator_model.pth')

# model evaluation
def calculate_perplexity(data, model, batch_size, block_size):
    model.eval()
    losses = []
    for i in range(0, len(data) - block_size, batch_size):
        x = torch.stack([data[j:j+block_size] for j in range(i, min(i+batch_size, len(data) - block_size))])
        y = torch.stack([data[j+1:j+block_size+1] for j in range(i, min(i+batch_size, len(data) - block_size))])
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits, loss = model(x, y)
            losses.append(loss.item())
        # early exit, since the whole token sets run indefinitely. 
        if i >= 500000:
            break
    model.train()
    perplexity = math.exp(np.mean(losses))
    return perplexity

# calculate perplexity for validation data
val_perplexity = calculate_perplexity(val_data, model, batch_size, block_size)
print(f'Validation Perplexity: {val_perplexity}')

0.827981 M parameters
step 0: train loss 4.5068, val loss 4.5078
step 100: train loss 2.3225, val loss 2.3329
step 200: train loss 2.1901, val loss 2.1981
step 300: train loss 2.0116, val loss 2.0214
step 400: train loss 1.8518, val loss 1.8667
step 500: train loss 1.7385, val loss 1.7450
step 600: train loss 1.6510, val loss 1.6606
step 700: train loss 1.5967, val loss 1.6106
step 800: train loss 1.5528, val loss 1.5685
step 900: train loss 1.5112, val loss 1.5227
step 1000: train loss 1.4844, val loss 1.5041
step 1100: train loss 1.4624, val loss 1.4755
step 1200: train loss 1.4391, val loss 1.4599
step 1300: train loss 1.4274, val loss 1.4444
step 1400: train loss 1.4046, val loss 1.4200
step 1500: train loss 1.3957, val loss 1.4137
step 1600: train loss 1.3781, val loss 1.3989
step 1700: train loss 1.3741, val loss 1.3920
step 1800: train loss 1.3539, val loss 1.3815
step 1900: train loss 1.3477, val loss 1.3688
step 2000: train loss 1.3434, val loss 1.3638
step 2100: train loss 1.

### Testing

In [4]:
# load the saved model
model = BigramLanguageModel()
model.load_state_dict(torch.load('lyric_generator_model.pth'))
model = model.to(device)

# function to generate a song
def generate_song(model, max_new_tokens=2000):
    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    song = decode(model.generate(context, max_new_tokens=max_new_tokens)[0].tolist())
    return song

# generate three different songs
song_1 = generate_song(m, max_new_tokens=2000)
song_2 = generate_song(m, max_new_tokens=2000)
song_3 = generate_song(m, max_new_tokens=2000)

# Print the generated songs
print("1st Song:\n", song_1)
print("\n2nd Song:\n", song_2)
print("\n3rd Song:\n", song_3)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [10]:
import unittest
import torch
import math

class TestBigramLanguageModel(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        global device
        device = torch.device('cpu')  # Force CPU usage for testing
        
        # Load the pre-trained model
        cls.model = BigramLanguageModel()
        model_path = '/kaggle/input/lyrics-generator-by-spotify-million-song-dataset/pytorch/default/1/lyric_generator_model.pth'
        cls.model.load_state_dict(torch.load(model_path, map_location=device))
        cls.model = cls.model.to(device)
        
        # Sample text for testing
        cls.text = "This is a sample text for testing purposes."
        
        # Tokenize text using the model's vocabulary
        cls.data, cls.stoi, cls.itos, cls.vocab_size, cls.encode, cls.decode = cls.text_tokenize(cls.text, cls.model)

        # Split data
        cls.train_data, cls.val_data = train_test_split(cls.data, 0.8)

    @staticmethod
    def text_tokenize(text, model):
        # Use the model's vocab size
        stoi = model.token_embedding_table.weight.size(0)
        itos = {i: ch for i, ch in enumerate(sorted(set(text)))}
        stoi = {ch: i for i, ch in enumerate(sorted(set(text)))}

        encode = lambda s: [stoi[c] for c in s if c in stoi]
        decode = lambda l: ''.join([itos[i] for i in l if i in itos])

        data = torch.tensor(encode(text), dtype=torch.long)

        return data, stoi, itos, len(stoi), encode, decode

    def test_model_initialization(self):
        """Test if the model initializes correctly."""
        self.assertIsInstance(self.model, BigramLanguageModel)
        # Adjust to match the actual model's vocabulary size (77 in this case)
        self.assertEqual(self.model.token_embedding_table.num_embeddings, 77)

    def test_forward_pass(self):
        """Test the forward pass of the model."""
        x, y = get_batch('train')
        logits, loss = self.model(x, y)
        self.assertEqual(logits.shape, (batch_size * block_size, self.model.token_embedding_table.num_embeddings))
        self.assertIsInstance(loss.item(), float)

    def test_generate(self):
        """Test the text generation functionality."""
        context = torch.zeros((1, 1), dtype=torch.long, device=device)
        generated = self.model.generate(context, max_new_tokens=10)
        self.assertEqual(generated.shape, (1, 11))

    def test_save_load_model(self):
        """Test saving and loading the model."""
        model_path = 'test_model.pth'
        torch.save(self.model.state_dict(), model_path)
        loaded_model = BigramLanguageModel()
        loaded_model.load_state_dict(torch.load(model_path, map_location=device))
        loaded_model = loaded_model.to(device)

        x, y = get_batch('train')
        logits_before, loss_before = self.model(x, y)
        logits_after, loss_after = loaded_model(x, y)
        self.assertTrue(torch.allclose(logits_before, logits_after))
        self.assertAlmostEqual(loss_before.item(), loss_after.item())

    def test_estimate_loss(self):
        """Test the loss estimation function."""
        losses = estimate_loss()
        self.assertIn('train', losses)
        self.assertIn('val', losses)
        self.assertIsInstance(losses['train'].item(), float)
        self.assertIsInstance(losses['val'].item(), float)

# Run the tests
unittest.main(argv=[''], verbosity=2, exit=False)


test_estimate_loss (__main__.TestBigramLanguageModel)
Test the loss estimation function. ... ok
test_forward_pass (__main__.TestBigramLanguageModel)
Test the forward pass of the model. ... ok
test_generate (__main__.TestBigramLanguageModel)
Test the text generation functionality. ... ok
test_model_initialization (__main__.TestBigramLanguageModel)
Test if the model initializes correctly. ... ok
test_save_load_model (__main__.TestBigramLanguageModel)
Test saving and loading the model. ... ok

----------------------------------------------------------------------
Ran 5 tests in 93.057s

OK
